In [11]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops
import os
import pickle
from opencc import OpenCC
# from transformers import RobertaForTokenClassification
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer , XLNetTokenizer , XLNetForPreTraining
from transformers import BertTokenizer , BertConfig , BertModel

import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining
from torch.autograd import Variable

from torch import nn

cc = OpenCC('t2s')


ImportError: cannot import name 'XLNetForPreTraining'

In [2]:
# train_df = pd.read_csv("./tbrain/tbrain_train_split.csv")
train_df = pd.read_csv("./tbrain/tbrain_train.csv")

train_df = train_df.fillna('None')

# test_df = pd.read_csv("./tbrain/tbrain_test_split.csv")
test_df = pd.read_csv("./tbrain/tbrain_test.csv")

test_df = test_df.fillna('None')

print(train_df.shape)
print(test_df.shape)

(4426, 4)
(491, 4)


In [3]:
def clean_string(content):
    content = content.replace('\n','').replace('\t','').replace(' ','').replace('\xa0','')
    content = re.sub("[●▼►★]", "",content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < 510:
            string = string + sentence
        else:
#             原本是空的代表sentences太常
            if string == '':
                n = 510
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li


In [4]:
train_content = train_df['full_content'].apply(clean_string).tolist()
test_content = test_df['full_content'].apply(clean_string).tolist()
all_content = train_content + test_content
print(len(all_content))

4917


In [8]:
pretrain_content = []

for content in all_content:
    split_contents = cut_sent(content)
    tmp = combine_sentence(split_contents)
    pretrain_content = pretrain_content + tmp

print(len(pretrain_content))   
        

12140


In [10]:
tokenizer = XLNetTokenizer.from_pretrained(pretrain_model_name)
model = XLNetForPreTraining.from_pretrained(pretrain_model_name)


NameError: name 'XLNetTokenizer' is not defined

In [ ]:
class PretrainDataset(Dataset):
    def __init__(self, input_dict):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        return inputid , tokentype , attentionmask
    
    def __len__(self):
        return len(self.input_ids)

In [ ]:
input_dict = tokenizer.batch_encode_plus(pretrain_content, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')

In [ ]:
BATCH_SIZE = 4
pretrainset = PretrainDataset(input_dict)
pretrainloader = DataLoader(pretrainset, batch_size=BATCH_SIZE)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
EPOCHS = 2
for epoch in range(EPOCHS):
    for data in pretrainloader:   
        tokens_tensors, segments_tensors, masks_tensors = [t.to(device) for t in data]
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, )
    model.save_pretrained('./pretrain_roberta_on_TBdata')
    print('epoch:', epoch+1)